# SQLAlchemy

SQLAlchemy - biblioteka skirta palengvinti darbą su duomenų bazėmis, komunikaciją Python <-> duomenų bazė. Ši biblioteka naudojama kaip ORM (angl. Object Relational Mapper) įrankis, kuris "išverčia" Python aprašytas klases į duomenų bazės lenteles ir automatiškai paverčia funkcijų kvietimus į SQL užklausas. SQLAlchemy biblioteka leidžia programuotojui dirbti neprisirišant prie konkrečios duomenų bazės, nes naudojama viena standartinė sąsaja darbui su daug skirtingų duomenų bazių. 

Kuriama programa <-> SQLAlchemy <-> DBAPI (angl. DataBase API) konkrečios duomenų bazės (MySQL, SQLite, PostgreSQL ir t.t.)

Prieš pradedant dirbti su SQLAlchemy ir duomenų bazėmis reikia sukurti varikliuką (angl. engine), kuris atliks komunikaciją su DBAPI funkcijomis. 

In [1]:
# sukuriamas variklis darbui su SQLite duomenų baze
from sqlalchemy import create_engine
engine = create_engine("sqlite:///:memory:", echo=True) # echo=True leis matyti visas SQL komandas

Norint nurodyti duomenų bazės lenteles ir jas atitinkančias Python klases naudojamas deklaratyvi sistema (angl. Declarative system). Šis sistema leidžia susieti lenteles su klasėmis. Tam naudojama Bazinė (angl. Base) klasė, kurios pagalba valdomas klasių ir lentelių sąsaja.

In [2]:
# sukuriama deklaratyvi bazinė klasė
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

SQLAlchemy biblioteka palaiko visus populiariausius duomenų tipus. Minimalūs reikalavimai kuriant modelius: \_\_tablename\_\_ ir bent vienas stulpelis.

In [3]:
# deklaratyvios klasės sukūrimas
from sqlalchemy import Column, Integer, String

class User(Base):
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)
    
    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (self.name, self.fullname, self.password)

Sukūrus User klasę per deklaratyvią sistemą, aprašėm informaciją apie lentelę, dar vadinamą metaduomenimis (angl. metadata). Sukurtas lentelės objektas yra didesnės kolekcijos MetaData narys. MetaData yra registras visų lentelių objektų ir jis atlieka jų sukūrimą duomenų bazėje.

In [4]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('password', String(), table=<users>), schema=None)

In [5]:
# sukuriama lentelė duomenų bazėje
Base.metadata.create_all(engine)

2018-02-08 23:48:29,266 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-02-08 23:48:29,270 INFO sqlalchemy.engine.base.Engine ()
2018-02-08 23:48:29,275 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-02-08 23:48:29,278 INFO sqlalchemy.engine.base.Engine ()
2018-02-08 23:48:29,283 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-02-08 23:48:29,284 INFO sqlalchemy.engine.base.Engine ()
2018-02-08 23:48:29,290 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2018-02-08 23:48:29,292 INFO sqlalchemy.engine.base.Engine ()
2018-02-08 23:48:29,297 INFO sqlalchemy.engine.base.Engine COMMIT


Dabar Users klasė yra susieta su duomenų bazės lentele.

In [13]:
# sukuriamas klasės Users objektas
tom_user = User(name="tom", fullname="Tomas Jonaitis", password="tomo_password")

In [14]:
print(tom_user.name, tom_user.fullname, tom_user.password)

tom Tomas Jonaitis tomo_password


Sukūrus norimas lenteles galima pradėti "šnekėtis" (kurti, redaguoti, trinti, ieškoti įrašų) su duomenų baze. Tai atliekama per sisijas (angl. session).

In [15]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [16]:
# duomenų perdavimas į sesiją
session.add(tom_user)

Kol kas duomenys nėra įrašyti į duomenų bazę, jie yra laukimo (angl. pending) būsenoje. Duomenys į lentelę bus surašyti tik iškvietus duomenų surašymo (angl. commit) komandą. Jei šiuo metu būtų iškviesta duomenų paieškos funkcija, tai laukimo būsenoje esantys duomenys būtų automatiškai surašyti į duomenų bazę. 

Biblioteka užtikrina, jog visos operacijos atliekamos sesijoje operuos tais pačiais duomenimis.

In [18]:
# atliekama duomenų paieška duomenų bazėje pagal name lauką
our_user = session.query(User).filter_by(name="tom").first()

print(our_user)
print(tom_user is our_user)

2018-02-09 00:05:08,865 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2018-02-09 00:05:08,868 INFO sqlalchemy.engine.base.Engine ('tom', 1, 0)
<User(name='tom', fullname='Tomas Jonaitis', password='tomo_password')>
True


In [19]:
# į sesiją surašomi nauji User objektai add_all([])
session.add_all([
    User(name="wendy", fullname="Wendy Williams", password="foobar"),
    User(name="mary", fullname="Mary Contrary", password="xxg527"),
    User(name="fred", fullname="Fred Flinstone", password="blah")])

In [20]:
# pakeičiamas vartotojo slaptažodis
tom_user.password = 'f8s7ccs'

Surašytus duomenis į sesiją galima pamatyti:

In [21]:
print(session.dirty)
print(session.new)

IdentitySet([<User(name='tom', fullname='Tomas Jonaitis', password='f8s7ccs')>])
IdentitySet([<User(name='wendy', fullname='Wendy Williams', password='foobar')>, <User(name='mary', fullname='Mary Contrary', password='xxg527')>, <User(name='fred', fullname='Fred Flinstone', password='blah')>])


In [22]:
# inicijuojamas duomenų surašymas į duomenų bazė
session.commit()

2018-02-09 00:11:45,315 INFO sqlalchemy.engine.base.Engine UPDATE users SET password=? WHERE users.id = ?
2018-02-09 00:11:45,318 INFO sqlalchemy.engine.base.Engine ('f8s7ccs', 2)
2018-02-09 00:11:45,322 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2018-02-09 00:11:45,323 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'foobar')
2018-02-09 00:11:45,326 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2018-02-09 00:11:45,327 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'xxg527')
2018-02-09 00:11:45,329 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2018-02-09 00:11:45,331 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flinstone', 'blah')
2018-02-09 00:11:45,334 INFO sqlalchemy.engine.base.Engine COMMIT


In [25]:
# duomenys surašyti duomenų bazėje, galima pamatyti sugeneruotas laukų reikšmes
print(tom_user.id)

2


# Duomenų atstatymas (angl. rolling back)